In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
from nltk.corpus import stopwords
import pickle
# nltk.download("stopwords")
sns.set_style('darkgrid')

In [2]:
df=pd.read_csv('zomato.csv')
df

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,080 40301477,Whitefield,Bar,NaN,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,+91 8197675843,Whitefield,Bar,NaN,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,NaN,Whitefield,Bar,NaN,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,080 49652769,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [4]:
df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [5]:
df["rate"].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [6]:
df["rate"] = df["rate"].fillna(df["rate"].mode()[0])
df['rate'].isnull().sum()

0

In [7]:
rate_hyphen = df.loc[df["rate"]=="-"].index
df.drop(rate_hyphen, axis = 0, inplace = True)
df["rate"].isnull().sum()

0

In [8]:
df["rate"].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [9]:
df["rate"].replace({"NEW": "2.5 /5"}, inplace=True)
df["rate"].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', '2.5 /5', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '3.2 /5', '2.6 /5', '4.5 /5', '4.3 /5',
       '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5', '4.9 /5',
       '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5', '2.1 /5',
       '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [10]:
df["rate"] = df["rate"].apply(lambda x : x.split("/")[0])
df["rate"] = df["rate"].apply(lambda x : float(x.split(" ")[0]))
df["rate"].unique()

array([4.1, 3.8, 3.7, 3.6, 4.6, 4. , 4.2, 3.9, 3.1, 3. , 3.2, 3.3, 2.8,
       4.4, 4.3, 2.5, 2.9, 3.5, 2.6, 3.4, 4.5, 2.7, 4.7, 2.4, 2.2, 2.3,
       4.8, 4.9, 2.1, 2. , 1.8])

In [11]:
df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                               0
votes                              0
phone                           1204
location                          21
rest_type                        227
dish_liked                     28009
cuisines                          45
approx_cost(for two people)      342
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [12]:
df.drop(["phone"], inplace=True, axis=1)

In [13]:
df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                               0
votes                              0
location                          21
rest_type                        227
dish_liked                     28009
cuisines                          45
approx_cost(for two people)      342
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [14]:
df = df.dropna(subset=["rest_type"])
df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                               0
votes                              0
location                           0
rest_type                          0
dish_liked                     27852
cuisines                          24
approx_cost(for two people)      319
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [15]:
df["dish_liked"] = df["dish_liked"].fillna("NONE")
df = df.dropna(subset=["cuisines"])

C:\Users\khush\AppData\Local\Temp/ipykernel_9852/3773455485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["dish_liked"] = df["dish_liked"].fillna("NONE")


In [16]:
df.isnull().sum()

url                              0
address                          0
name                             0
online_order                     0
book_table                       0
rate                             0
votes                            0
location                         0
rest_type                        0
dish_liked                       0
cuisines                         0
approx_cost(for two people)    314
reviews_list                     0
menu_item                        0
listed_in(type)                  0
listed_in(city)                  0
dtype: int64

In [17]:
df.rename(columns={"approx_cost(for two people)": "cost"}, inplace=True)

In [18]:
df = df.dropna(subset= ["cost"])
df.isnull().sum()

url                0
address            0
name               0
online_order       0
book_table         0
rate               0
votes              0
location           0
rest_type          0
dish_liked         0
cuisines           0
cost               0
reviews_list       0
menu_item          0
listed_in(type)    0
listed_in(city)    0
dtype: int64

In [19]:
df_recommendation = df.copy()
df_recommendation = df_recommendation.drop(["menu_item"], axis=1)
df_recommendation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51083 entries, 0 to 51716
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              51083 non-null  object 
 1   address          51083 non-null  object 
 2   name             51083 non-null  object 
 3   online_order     51083 non-null  object 
 4   book_table       51083 non-null  object 
 5   rate             51083 non-null  float64
 6   votes            51083 non-null  int64  
 7   location         51083 non-null  object 
 8   rest_type        51083 non-null  object 
 9   dish_liked       51083 non-null  object 
 10  cuisines         51083 non-null  object 
 11  cost             51083 non-null  object 
 12  reviews_list     51083 non-null  object 
 13  listed_in(type)  51083 non-null  object 
 14  listed_in(city)  51083 non-null  object 
dtypes: float64(1), int64(1), object(13)
memory usage: 6.2+ MB


In [20]:
restaurants = list(df_recommendation["name"].unique())
df_recommendation["Mean Rating"] = 0 
for i in range(len(restaurants)):
    df_recommendation["Mean Rating"][df_recommendation["name"] == restaurants[i]] = df_recommendation["rate"][df_recommendation["name"] == restaurants[i]].mean()
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
df_recommendation[["Mean Rating"]] = scaler.fit_transform(df_recommendation[["Mean Rating"]]).round(2)
df_recommendation[["name", "rate", "Mean Rating"]].head()

C:\Users\khush\AppData\Local\Temp/ipykernel_9852/4047312019.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recommendation["Mean Rating"][df_recommendation["name"] == restaurants[i]] = df_recommendation["rate"][df_recommendation["name"] == restaurants[i]].mean()
C:\Users\khush\AppData\Local\Temp/ipykernel_9852/4047312019.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recommendation["Mean Rating"][df_recommendation["name"] == restaurants[i]] = df_recommendation["rate"][df_recommendation["name"] == restaurants[i]].mean()


,name,rate,Mean Rating
0,Jalsa,4.1,3.99
1,Spice Elephant,4.1,3.97
2,San Churro Cafe,3.8,3.58
3,Addhuri Udupi Bhojana,3.7,3.45
4,Grand Village,3.8,3.58


In [21]:
df_recommendation["review_list"] = df_recommendation["reviews_list"].str.lower()


In [22]:
PUNCT_TO_REMOVE = string.punctuation
def removePunctuation(text):
    return text.translate(str.maketrans("", "", PUNCT_TO_REMOVE))
df_recommendation["reviews_list"] = df_recommendation["review_list"].apply(lambda text: removePunctuation(text))

In [23]:
def removeURLs(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
df_recommendation["reviews_list"] = df_recommendation["reviews_list"].apply(lambda text: removeURLs(text))

In [24]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
df_recommendation["reviews_list"] = df_recommendation["reviews_list"].apply(lambda text: remove_stopwords(text))

In [25]:
df_recommendation

,url,address,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,listed_in(type),listed_in(city),Mean Rating,review_list
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,rated 40 ratedn beautiful place dine inthe int...,Buffet,Banashankari,3.99,"[('rated 4.0', 'rated\n a beautiful place to ..."
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,rated 40 ratedn dinner family turned good choo...,Buffet,Banashankari,3.97,"[('rated 4.0', 'rated\n had been here for din..."
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,rated 30 ratedn ambience good enough pocket fr...,Buffet,Banashankari,3.58,"[('rated 3.0', ""rated\n ambience is not that ..."
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,rated 40 ratedn great food proper karnataka st...,Buffet,Banashankari,3.45,"[('rated 4.0', ""rated\n great food and proper..."
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,rated 40 ratedn good restaurant neighbourhood ...,Buffet,Banashankari,3.58,"[('rated 4.0', 'rated\n very good restaurant ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6,27,Whitefield,Bar,NONE,Continental,"1,500",rated 50 ratedn food service incomparably exce...,Pubs and bars,Whitefield,3.32,"[('rated 5.0', ""rated\n food and service are ..."
51713,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,2.5,0,Whitefield,Bar,NONE,Finger Food,600,,Pubs and bars,Whitefield,1.90,[]
51714,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,2.5,0,Whitefield,Bar,NONE,Finger Food,"2,000",,Pubs and bars,Whitefield,1.90,[]
51715,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3,236,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500",rated 40 ratedn nice friendly place staff awes...,Pubs and bars,Whitefield,4.23,"[('rated 4.0', 'rated\n nice and friendly pla..."


In [26]:
df_recommendation.info()
df_recommendation["rest_type"].unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51083 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              51083 non-null  object 
 1   address          51083 non-null  object 
 2   name             51083 non-null  object 
 3   online_order     51083 non-null  object 
 4   book_table       51083 non-null  object 
 5   rate             51083 non-null  float64
 6   votes            51083 non-null  int64  
 7   location         51083 non-null  object 
 8   rest_type        51083 non-null  object 
 9   dish_liked       51083 non-null  object 
 10  cuisines         51083 non-null  object 
 11  cost             51083 non-null  object 
 12  reviews_list     51083 non-null  object 
 13  listed_in(type)  51083 non-null  object 
 14  listed_in(city)  51083 non-null  object 
 15  Mean Rating      51083 non-null  float64
 16  review_list      51083 non-null  object 
dtypes: float64(2

array(['Casual Dining', 'Cafe, Casual Dining', 'Quick Bites',
       'Casual Dining, Cafe', 'Cafe', 'Quick Bites, Cafe',
       'Cafe, Quick Bites', 'Delivery', 'Mess', 'Dessert Parlor',
       'Bakery, Dessert Parlor', 'Pub', 'Bakery', 'Takeaway, Delivery',
       'Fine Dining', 'Beverage Shop', 'Sweet Shop', 'Bar',
       'Beverage Shop, Quick Bites', 'Confectionery',
       'Quick Bites, Beverage Shop', 'Dessert Parlor, Sweet Shop',
       'Bakery, Quick Bites', 'Sweet Shop, Quick Bites', 'Kiosk',
       'Food Truck', 'Quick Bites, Dessert Parlor',
       'Beverage Shop, Dessert Parlor', 'Takeaway', 'Pub, Casual Dining',
       'Casual Dining, Bar', 'Dessert Parlor, Beverage Shop',
       'Quick Bites, Bakery', 'Dessert Parlor, Quick Bites',
       'Microbrewery, Casual Dining', 'Lounge', 'Bar, Casual Dining',
       'Food Court', 'Cafe, Bakery', 'Dhaba', 'Quick Bites, Sweet Shop',
       'Microbrewery', 'Food Court, Quick Bites', 'Pub, Bar',
       'Casual Dining, Pub', 'Lounge, Ba

In [27]:
df_percent = df_recommendation.sample(frac=0.5)
df_percent = df_percent.set_index(['name'], inplace=False)
# df_percent["overall_review"] = df_percent[]
indices = pd.Series(df_percent.index)

In [28]:
# df_percent["dish_liked"] = df_percent["dish_liked"].apply(lambda x: )
df_percent["overall_review"] = df_percent["dish_liked"] + df_percent["cuisines"] + df_percent["reviews_list"] 
df_percent["overall_review"]

name
Green Chilly                 NONENorth Indian, Chineserated 30 ratedn big p...
Pin Me Down                  Paneer Tikka, Pasta, Chilli Chicken, Pizza, Mo...
Foodies Family Restaurant    NONENorth Indian, Chineserated 20 ratedn order...
Sufi Restaurant & Cafe       NONEIranian, Mughlai, Chineserated 30 ratedn n...
Woodee Pizza                 Pizza, Lemonade, Chilli Cheese Fries, Thick Sh...
                                                   ...                        
WOW! Momo                    Burgers, Chocolate Momos, Noodles, Cheese Momo...
Chaitanya Cafe               Coffee, Pav Bhaji, Masala Dosa, Vada SambarSou...
BTM Bawarchi Biryani         Chicken BiryaniBiryani, North Indian, Chineser...
Sri Krishna Bhavan Sweets    Fruit Juices, ChaatMithai, Street Foodrated 40...
Fitnest                      NONEHealthy Food, North Indianrated 30 ratedn ...
Name: overall_review, Length: 25542, dtype: object

In [29]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['overall_review'])

In [30]:
# tfidf_cu = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
# tfidf_cu_matrix = tfidf_cu.fit_transform(df_percent["cuisines"])

In [31]:
# tfidf_rest_type = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
# tfidf_rest_type_matrix = tfidf_rest_type.fit_transform(df_percent["rest_type"])

In [32]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [33]:
# cosine_similarities_cu = linear_kernel(tfidf_cu_matrix, tfidf_cu_matrix)

In [34]:
# cosine_similarities_rest_type = linear_kernel(tfidf_rest_type_matrix, tfidf_rest_type_matrix)

In [35]:
print(cosine_similarities)

[[1.         0.72688348 0.0221915  ... 0.14225358 0.38335185 0.03039366]
 [0.72688348 1.         0.00823334 ... 0.12303743 0.48173631 0.01053108]
 [0.0221915  0.00823334 1.         ... 0.05410548 0.00238847 0.02649992]
 ...
 [0.14225358 0.12303743 0.05410548 ... 1.         0.06864656 0.10549827]
 [0.38335185 0.48173631 0.00238847 ... 0.06864656 1.         0.01463897]
 [0.03039366 0.01053108 0.02649992 ... 0.10549827 0.01463897 1.        ]]


In [36]:
def recommend(name, cosine_similarities = cosine_similarities):
    recommend_restaurant = []
    idx = indices[indices == name].index[0]
    print("index: ",idx)
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    print(score_series)
    top30_indexes = list(score_series.iloc[0:31].index)
    print(top30_indexes)
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost', "location", "dish_liked", "address"]][df_percent.index == each].sample()))
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
#     df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    df_new = df_new.head(10)
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    return df_new

In [37]:
recommended = recommend("Pai Vihar")


 # recommended

index:  2078
2078     1.000000
16131    0.695090
21659    0.164459
15277    0.164459
13297    0.164459
           ...   
22315    0.000000
1942     0.000000
22311    0.000000
7295     0.000000
17684    0.000000
Length: 25542, dtype: float64
[2078, 16131, 21659, 15277, 13297, 4087, 14464, 136, 2002, 14031, 16462, 12557, 12990, 15535, 23167, 23320, 10199, 10425, 15468, 15203, 21529, 22884, 4681, 5060, 16844, 19998, 19427, 24559, 15040, 8779, 21851]
TOP 8 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS: 


In [38]:
import json
restaurant_list = []
for rest in recommended.index.values:
    res = {}
    print(rest)
    res["name"] = rest
    res["location"] = recommended.at[rest,'location']
    res["cuisines"] = [cu.strip() for cu in recommended.at[rest,"cuisines"].split(",")]
    res["Mean Rating"] = recommended.at[rest,"Mean Rating"]
    res["cost"] = recommended.at[rest,"cost"]
    res["dish_liked"] = [dish.strip() for dish in recommended.at[rest,"dish_liked"].split(", ")]
    res["address"] = recommended.at[rest,"address"]
    restaurant_list.append(res)

print(json.dumps(restaurant_list[0], indent = 4))

Mumbai Spice
Jodhpur Jaika
Sai Canteen
Shiv Sagar
Sagar Fast Food
Ayodhya Food Line
Hunger Lust
Desi Feast
{
    "name": "Mumbai Spice",
    "location": "Malleshwaram",
    "cuisines": [
        "North Indian",
        "Chinese"
    ],
    "Mean Rating": 2.94,
    "cost": "400",
    "dish_liked": [
        "Paratha",
        "Vada Pav",
        "Thali",
        "Lassi",
        "Manchurian"
    ],
    "address": "18/2, Temple Road, 10th Cross, Malleshwaram, Bangalore"
}


In [56]:
recommend("New Shanthi Sagar")

index:  843
843      1.000000
5252     0.388737
19691    0.388737
8376     0.388737
10677    0.388737
           ...   
15295    0.000000
1188     0.000000
23461    0.000000
15325    0.000000
4616     0.000000
Length: 25542, dtype: float64
[843, 5252, 19691, 8376, 10677, 23157, 9993, 17476, 23472, 1147, 19565, 24719, 9144, 1018, 11702, 17215, 8901, 7098, 18073, 14534, 6312, 3047, 8956, 24233, 13341, 729, 9054, 5728, 693, 5352, 16733]
TOP 9 RESTAURANTS LIKE New Shanthi Sagar WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost,location,dish_liked,address
New Shanthi Sagar,"South Indian, North Indian, Chinese, Street Food",2.52,300,BTM,NONE,"01, 100 Feet Ring Road, 1st Phase, 2nd Stage, ..."
Udupi Biriyani Point,"North Indian, South Indian",1.90,190,Bannerghatta Road,NONE,"Behind Shanthinikethan School, Bilekahalli, Ba..."
Navya Andhra Style Restaurant,"North Indian, South Indian",3.19,200,Hosur Road,NONE,"SG Palya, 1st Main, Second Cross, Near Sriniva..."
Sri Srinivas Park Hotel,"South Indian, Fast Food",1.90,300,BTM,NONE,"147 Ejipura Road, Near Green Vine Hospital BTM..."
Indian Spicy Food,"North Indian, South Indian",2.81,200,Old Madras Road,NONE,"Opposite Metro 135, Near IndusInd Bank, Old Ma..."
Nonveg Dreams,"Biryani, North Indian",1.90,500,Koramangala 2nd Block,NONE,"136, 3rd Floor, Sidartha Colony, 3rd Croos Kor..."
Gowdru Mane Bhaduta,"North Indian, South Indian",2.81,400,Rajajinagar,NONE,"37, West Of Chord Road, 1st Main, Shivanagar, ..."
BTM,"North Indian, South Indian",1.90,400,BTM,NONE,"Kolliyil, Kolliylhouse, Madiwala, Margi Nagar ..."
LG Park Restaurant,"North Indian, South Indian",3.06,800,Whitefield,NONE,"KR Extension, Near KEB Circle, Hoskote, Whitef..."


In [40]:
# recommend("Truffles", cosine_similarities_cu)

In [41]:
# recommend("Gowdru Hotel", cosine_similarities_rest_type)

In [42]:
# df_recommendation[["name", "cuisines"]][df_recommendation["name"] == "The Pizza Bakery"]

In [43]:
# df_percent.index

In [44]:
import pickle
filename= "./pickle_files/model.pkl"
pickle.dump(cosine_similarities, open(filename, "wb"))

In [45]:
import pickle
loaded_cosine_similarities = pickle.load(open("./pickle_files/model.pkl", "rb"))
# recommend("Truffles", loaded_cosine_similarities, )

In [46]:
recommend("Biergarten", loaded_cosine_similarities)

index:  2864
2864     1.000000
23402    0.801308
23674    0.421322
4090     0.394895
9519     0.383857
           ...   
5149     0.000000
5146     0.000000
15437    0.000000
15436    0.000000
21071    0.000000
Length: 25542, dtype: float64
[2864, 23402, 23674, 4090, 9519, 22510, 9203, 4170, 7950, 2045, 3731, 14020, 11235, 20464, 20996, 191, 4032, 3092, 1992, 11090, 6071, 25480, 13771, 19398, 3066, 17907, 24784, 19883, 20663, 14436, 23725]
TOP 5 RESTAURANTS LIKE Biergarten WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost,location,dish_liked,address
Prost Brew Pub,"American, Continental, North Indian, Salad",4.48,"1,800",Koramangala 4th Block,"Mango Cider, Cajun Potato, Nachos, Salads, Piz...","749, 10th Main, 80 Feet Road, Koramangala 4th ..."
Simon Says Brew Works,"Continental, American, Italian, Modern Indian,...",4.48,"1,400",Koramangala 5th Block,"Beer, Onion Pakoda, Long Island Iced Tea, Peri...","105, 3rd Floor, 1st A Cross Road, Jyothi Nivas..."
Brewklyn Microbrewery,"American, South Indian, Thai, Pizza, Italian",4.38,"1,400",Kalyan Nagar,"Pizza, Beef Chilli, Craft Beer, Apple Crumble,...","30, B.R.Plaza, 4th Floor, 4th Cross, CMR Road,..."
Cinnamon,"North Indian, Asian, Continental",3.62,"1,000",Seshadripuram,NONE,"211, S C Road, Seshadripuram, Bangalore"
Levitate Brewery and Kitchen,"Finger Food, North Indian, Continental",2.23,"1,500",JP Nagar,"Wheat Beer, Lager, Paneer Lollipop, Rustica Pi...","57, 1st Main Rd, Sarakki Industrial Layout, 3r..."


In [47]:
# pickle.dump(df_percent, open("model_index.pkl", "wb"))
pickle.dump(df_percent, open("./pickle_files/model_data_set.pkl","wb"))
pickle.dump(indices, open("./pickle_files/model_indices.pkl", "wb"))

In [48]:
loaded_df_percent = pickle.load(open("./pickle_files/model_data_set.pkl", "rb"))
loaded_indices = pickle.load(open("./pickle_files/model_indices.pkl", "rb"))
loaded_df_percent.index

Index(['Green Chilly', 'Pin Me Down', 'Foodies Family Restaurant',
       'Sufi Restaurant & Cafe', 'Woodee Pizza', 'Chaatimes', 'Sandhya Sagar',
       'Dine in Restaurant', 'Shake The Shake', 'HSR Agarval Bhavan',
       ...
       'Ragoo's Kitchen', 'Momo Time', 'Rivayat', 'Anjappar', 'Maddy's Cafe',
       'WOW! Momo', 'Chaitanya Cafe', 'BTM Bawarchi Biryani',
       'Sri Krishna Bhavan Sweets', 'Fitnest'],
      dtype='object', name='name', length=25542)

In [49]:
# def recommendLoaded(name, indices,cosine_similarities):
#     recommend_restaurant = []
# #     indices = pd.Series(indices)
#     idx = indices[indices == name].index[0]
#     print("index: ",idx)
#     score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
#     print(score_series)
#     top30_indexes = list(score_series.iloc[0:31].index)
#     print(top30_indexes)
#     for each in top30_indexes:
#         recommend_restaurant.append(list(loaded_df_percent.index)[each])
#     df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    
#     for each in recommend_restaurant:
#         df_new = df_new.append(pd.DataFrame(loaded_df_percent[['cuisines','Mean Rating', 'cost']][loaded_df_percent.index == each].sample()))
#     df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
#     df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
#     print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
#     return df_new

In [50]:
# recommendLoaded("Truffles", loaded_indices, loaded_df_percent)

In [51]:
# from flask import Flask, jsonify, request 
# app = Flask(__name__)
# @app.route("/ping", methods=["GET"])
# def home():
#     if(request.method == "GET"):
#         data = "pong"
#         return jsonify({"data": data})

# if __name__ == "__main__":
#     from werkzeug.serving import run_simple
#     run_simple("localhost", 9000, app)
# #     app.run(debug = True)